# Read Overpass-API to Python Pandas Dataframe

The [Overpass API](http://wiki.openstreetmap.org/wiki/Overpass_API) provides access to the data behind the Openstreetmaps Map Data.
The [Overpass-Turbo](http://overpass-turbo.eu/) is the easyiest way to test requests and get the correct code to ask the database. Example: Fountains with wikidata Q-Numbers as per https://github.com/water-fountains/import2wikidata/issues/15

In [1]:
from datetime import datetime as dt
dtFmt = "%y%m%d_%H%M%S"
print (dt.now().strftime(dtFmt))
import pandas as pd
import requests
import json
pd.set_option('display.max_columns', 200)

191201_075951


In [2]:
# Get yours at: http://boundingbox.klokantech.com/
# Rome ll: 41.8020,12.3170 ; ur: 41.9973,12.6219
bbox = [12.3170,41.8020,12.6219,41.9973]

# Links unten
minLat = bbox[1]
minLon = bbox[0]

# Rechts oben
maxLat = bbox[3]
maxLon = bbox[2]

### Construct the Overpass Query String

Request from [Overpass-Turbo](http://overpass-turbo.eu/)

In [3]:
def makeOsmQuery(minLat, minLon, maxLat, maxLon):
    bbox_string = '(%s,%s,%s,%s)' % (minLat, minLon, maxLat, maxLon)
    
    compactOverpassQLstring = f'''
    
    [out:json][timeout:60];
    
    (
        node[man_made=drinking_fountain][wikidata]{bbox_string};
        node[amenity=drinking_water][wikidata]{bbox_string};
        node[amenity=fountain][wikidata]{bbox_string};
    );'''
    
    
    #node[man_made=water_tap]{bbox_string};node[natural=spring]{bbox_string};node[amenity=watering_place]{bbox_string};
    #node[amenity=water_point]{bbox_string};node[man_made=water_well]{bbox_string};
    compactOverpassQLstring += 'out ;'
    return compactOverpassQLstring

In [4]:
compactOverpassQLstring = makeOsmQuery(minLat, minLon, maxLat, maxLon)
compactOverpassQLstring

'\n    \n    [out:json][timeout:60];\n    \n    (\n        node[man_made=drinking_fountain][wikidata](41.802,12.317,41.9973,12.6219);\n        node[amenity=drinking_water][wikidata](41.802,12.317,41.9973,12.6219);\n        node[amenity=fountain][wikidata](41.802,12.317,41.9973,12.6219);\n    );out ;'

In [5]:
osmrequest = {'data': compactOverpassQLstring}
osmurl = 'https://overpass-api.de/api/interpreter'

# Ask the API
osm = requests.get(osmurl, params=osmrequest)

### Reformat the JSON to fit in a Pandas Dataframe

The JSON can't be directyl imported to a Pandas Dataframe:

Thanks to [unutbu from stackoverflow.com](http://stackoverflow.com/questions/24848416/expected-string-or-unicode-when-reading-json-with-pandas) for fiddling this out!

In [6]:
osmdata = osm.json()
# print out the json
osmdata

{'version': 0.6,
 'generator': 'Overpass API 0.7.55.1009 5e627b63',
 'osm3s': {'timestamp_osm_base': '2019-12-01T07:31:02Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 259261742,
   'lat': 41.9315211,
   'lon': 12.4856511,
   'tags': {'amenity': 'fountain',
    'name': "Fontana dell'Acqua Acetosa",
    'tourism': 'attraction',
    'wikidata': 'Q3747370',
    'wikipedia': "it:Fontana dell'Acqua Acetosa"}},
  {'type': 'node',
   'id': 279788989,
   'lat': 41.8814447,
   'lon': 12.519195,
   'tags': {'amenity': 'drinking_water',
    'bottle': 'yes',
    'dog': 'limited',
    'fountain': 'nasone',
    'wheelchair': 'yes',
    'wikidata': 'Q74574057'}},
  {'type': 'node',
   'id': 281348354,
   'lat': 41.8983031,
   'lon': 12.481382,
   'tags': {'amenity': 'fountain',
    'architect': 'Jacopo del Conte',
    'drinking_water': 'yes',
    'name': 'Fontana del Facchino',
   

In [7]:
osmelements = osmdata['elements']
for dct in osmelements:
    for key, val in dct['tags'].items():
        dct[key] = val
    del dct['tags']

### Now put everything to the Pandas Dataframe

In [8]:
osmdf = pd.DataFrame(osmelements)

### Look at the whole Dataframe

In [9]:
osmdf.head(5)

,type,id,lat,lon,amenity,name,tourism,wikidata,wikipedia,bottle,dog,fountain,wheelchair,architect,drinking_water,start_date,alt_name,flickr,fountain:type,name:de,name:en,name:es,name:fr,name:it,name:ru,name:zh,note,survey:date,name:ja,name:sv,historic,addr:postcode,addr:street,description
0,node,259261742,41.931521,12.485651,fountain,Fontana dell'Acqua Acetosa,attraction,Q3747370,it:Fontana dell'Acqua Acetosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,279788989,41.881445,12.519195,drinking_water,NaN,NaN,Q74574057,NaN,yes,limited,nasone,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sculpture,281348354,41.898303,12.481382,fountain,Fontana del Facchino,NaN,Q3968656,it:Statua del Facchino,NaN,NaN,NaN,yes,Jacopo del Conte,yes,1587..1598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,1277743956,41.904390,12.494452,fountain,Fontana dell'Acqua Felice,NaN,Q983774,it:Fontana dell'Acqua Felice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fontana del Mosè,https://www.flickr.com/photos/122687277@N03/36...,backed,Mosesbrunnen,Fountain of Moses,Fuente del Agua Feliz,Fontana dell'Acqua Felice,Fontana dell'Acqua Felice,Фонтан Моисея,摩西喷泉或快乐水道桥喷泉,"if drinkable, insert tag drinkable=yes",2017-08-23,NaN,NaN,NaN,NaN,NaN,NaN
4,node,1326896537,41.904297,12.488767,fountain,Fontana delle api,NaN,Q857540,it:Fontana delle Api,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bienen-Brunnen,Fountain of the Bees,NaN,NaN,NaN,Фонтан Пчёл,蜂蜜噴泉,NaN,NaN,蜂の噴水,Bifontänen,NaN,NaN,NaN,NaN


In [10]:
osmdfShrt =  osmdf[['lat','lon','wikidata']]

In [11]:
osmdfShrt.tail(5)

,lat,lon,wikidata
6,41.888040,12.467327,Q3747364
7,41.866839,12.542013,Q3908713
8,41.907764,12.479684,Q1085738
9,41.886857,12.504790,Q3747378
10,41.891686,12.469798,Q3747421


## Export to CSV

In [12]:
osmCsvFileName = "osmFountainsWithWikidataReferenceRome"+dt.now().strftime(dtFmt)+".csv"
osmdfShrt.to_csv(osmCsvFileName, index=False)



In [13]:
print("wrote '"+osmCsvFileName+"' with "+str(len(osmdf))+" lines")

wrote 'osmFountainsWithWikidataReferenceRome191201_080632.csv' with 11 lines
